In [14]:
# Alexander Ly 027744520
# CECS 451 Sec 01
# Assignment 4 - MCMC
# March 16, 2024

import sorobn as hh
import pandas as pd
import random

# C = Cloudy
# S = Sprinkler
# R = Rain
# W = Wet Grass

bn = hh.BayesNet(
    ('C', ['S', 'R']),
    ('S', 'W'),
    ('R', 'W'))

bn.P['C'] = pd.Series({True: 0.5, False:0.5})
bn.P['S'] = pd.Series({
    (True, True): 0.1, (True, False): 0.9,
    (False, True): 0.5, (False, False): 0.5})
bn.P['R'] = pd.Series({
    (True, True): 0.8, (True, False): 0.2,
    (False, True): 0.2, (False, False): 0.8})
bn.P['W'] = pd.Series({
    (True, True, True): 0.99, (True, True, False): 0.01,
    (True, False, True): 0.9, (True, False, False): 0.1,
    (False, True, True): 0.95, (False, True, False): 0.05,
    (False, False, True): 0.05, (False, False, False): 0.95})
bn.prepare()
bn.query('R', event={'C': False, 'S': False, 'W': True})
#bn.query('C', event={'S': False, 'R': False})

R
False    0.181818
True     0.818182
Name: P(R), dtype: float64

In [15]:
# Part A
print("Part A. The sampling probabilities")
print("P(C|-s,r) = <{:.4f}, {:.4f}>".format(0.878049, 0.121951))
print("P(C|-s,-r) = <{:.4f}, {:.4f}>".format(0.310345, 0.689655))
print("P(R|c,-s,w) = <{:.4f}, {:.4f}>".format(0.986301, 0.013699))
print("P(R|-c,-s,w) = <{:.4f}, {:.4f}>".format(0.818182, 0.181818))

Part A. The sampling probabilities
P(C|-s,r) = <0.8780, 0.1220>
P(C|-s,-r) = <0.3103, 0.6897>
P(R|c,-s,w) = <0.9863, 0.0137>
P(R|-c,-s,w) = <0.8182, 0.1818>


In [16]:
# Calculations 
prob_C_given_nots_r = {True: 0.878049, False: 0.121951}
prob_R_given_c_nots_w = {True: 0.986301, False: 0.013699}
prob_C_given_nots_notr = {True: 0.310345, False: 0.689655}
prob_R_given_notc_nots_w = {True: 0.818182, False: 0.181818}

# Create Matrix for values
matrix = [[0, 0, 0, 0],[0, 0, 0, 0],[0, 0, 0, 0],[0, 0, 0, 0]]

matrix[0][0] = (0.5 * prob_C_given_nots_r[True]) + (0.5 * prob_R_given_c_nots_w[True])
matrix[0][1] = 0.5 * prob_R_given_c_nots_w[False]
matrix[0][2] = 0.5 * prob_C_given_nots_r[False]
matrix[0][3] = 0

matrix[1][0] = 0.5 * prob_R_given_c_nots_w[True]
matrix[1][1] = (0.5 * prob_C_given_nots_notr[True]) + (0.5 * prob_R_given_c_nots_w[False])
matrix[1][2] = 0
matrix[1][3] = 0.5 * prob_C_given_nots_notr[False]

matrix[2][0] = 0.5 * prob_C_given_nots_r[True]
matrix[2][1] = 0
matrix[2][2] = (0.5 * prob_C_given_nots_r[False]) + (0.5 * prob_R_given_notc_nots_w[True])
matrix[2][3] = 0.5 * prob_R_given_notc_nots_w[False]

matrix[3][0] = 0
matrix[3][1] = 0.5 * prob_C_given_nots_notr[True]
matrix[3][2] = 0.5 * prob_R_given_notc_nots_w[True]
matrix[3][3] = (0.5 * prob_C_given_nots_notr[False]) + (0.5 * prob_R_given_notc_nots_w[False])

In [17]:
# Part B
print("Part B. The transition probability matrix")
print("     S1     S2     S3     S4")
print("S1", "{:.4f}".format(matrix[0][0]), "{:.4f}".format(matrix[0][1]), "{:.4f}".format(matrix[0][2]), "{:.4f}".format(matrix[0][3]))
print("S2", "{:.4f}".format(matrix[1][0]), "{:.4f}".format(matrix[1][1]), "{:.4f}".format(matrix[1][2]), "{:.4f}".format(matrix[1][3]))
print("S3", "{:.4f}".format(matrix[2][0]), "{:.4f}".format(matrix[2][1]), "{:.4f}".format(matrix[2][2]), "{:.4f}".format(matrix[2][3]))
print("S4", "{:.4f}".format(matrix[3][0]), "{:.4f}".format(matrix[3][1]), "{:.4f}".format(matrix[3][2]), "{:.4f}".format(matrix[3][3]))

Part B. The transition probability matrix
     S1     S2     S3     S4
S1 0.9322 0.0068 0.0610 0.0000
S2 0.4932 0.1620 0.0000 0.3448
S3 0.4390 0.0000 0.4701 0.0909
S4 0.0000 0.1552 0.4091 0.4357


In [18]:
# Part C

# Get value for P(C|-s,w)
prob_C_given_nots_w = bn.query('C', event={'S': False, 'W': True})
prob_C_given_nots_w_True = prob_C_given_nots_w[True]
prob_C_given_nots_w_False = prob_C_given_nots_w[False]
exact_prob_C_given_nots_w = {True: prob_C_given_nots_w_True, False: prob_C_given_nots_w_False}

# Output the exact value that we got from above
print("Part C. The probability for the query P(C|-s,w)")
print("Exact probability: <{:.4f}, {:.4f}>".format(prob_C_given_nots_w_True, prob_C_given_nots_w_False))

# For loop for each 10^n 
for i in range(3, 7): 
    # Set n equal to the power of each range
    n = 10**i  
    # Create dictionary to keep track of each time state is visited
    amount = {'S1': 0, 'S2': 0, 'S3': 0, 'S4': 0}
    # Start with a random state
    start_state = random.choice(['S1', 'S2', 'S3', 'S4'])

    # For loop to go through each given n
    for j in range(n):
        # Increment the count by 1 for the state that was chosen
        amount[start_state] += 1

        # Based on the chosen state, select the row correlated with it
        if start_state == 'S1':
            row = 0
        elif start_state == 'S2':
            row = 1
        elif start_state == 'S3':
            row = 2
        else:
            row = 3

        # Determine the row and get the probabilities for that row based on the current state
        current_row = matrix[row]

        # Randomly select the next state based on the transition probabilities
        next_state = random.choices(['S1', 'S2', 'S3', 'S4'], weights=current_row, k=1)[0]


        # change starting state to the state that was selected in previous 
        start_state = next_state

        
    # Total the true and false predicted probabilities
    pred_prob_C_given_not_s_w_true = (amount['S1'] + amount['S2']) / n
    pred_prob_C_given_not_s_w_false = (amount['S3'] + amount['S4']) / n

    # Get the error
    error = abs((exact_prob_C_given_nots_w[True] - pred_prob_C_given_not_s_w_true) / exact_prob_C_given_nots_w[True]) * 100
    
    # Output predicted value with error percentage for given n
    print(f"n = 10^{i}: <{pred_prob_C_given_not_s_w_true:.4f}, {pred_prob_C_given_not_s_w_false:.4f}>, error = {error:.2f}%")

Part C. The probability for the query P(C|-s,w)
Exact probability: <0.8566, 0.1434>
n = 10^3: <0.8260, 0.1740>, error = 3.57%
n = 10^4: <0.8505, 0.1495>, error = 0.71%
n = 10^5: <0.8547, 0.1453>, error = 0.22%
n = 10^6: <0.8566, 0.1434>, error = 0.00%
